In [43]:
import numpy as np
import torch
import plotly.express as px
import plotly
plotly.offline.init_notebook_mode(connected = True)

## 3.1 Автоматическое дифференцирование в `torch`

3.1.1 Воспользовавшись классами `Neuron` и `SquaredLoss` из задачи 2.4.1 и автоматическим дифференцированием, которое предоставляет `torch`, решить задачу регрессии. Для оптимизации использовать стохастический градиетный спуск.

In [44]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)

In [45]:
class Neuron:
    def __init__(self, n_inputs):
        self.weights = torch.randn(n_inputs, 1, requires_grad=True)
        self.bias = torch.randn(1, 1, requires_grad=True)

    def forward(self, inputs):
        return inputs @ self.weights + self.bias

    def backward(self):
        self.dweights = self.weights.grad
        self.dbias = self.bias.grad


In [46]:
class SquaredLoss:
    def forward(self, y_pred, y_true):
        self.loss = (y_pred - y_true).pow(2).mean()
        return self.loss

    def backward(self):
        self.loss.backward()


In [47]:
neuron = Neuron(4)
loss = SquaredLoss()

for epoch in range(100):
    for x_example, y_example in zip(X, y):
        y_pred = neuron.forward(x_example)
        loss.forward(y_pred, y_example)
        loss.backward()
        neuron.backward()
        with torch.no_grad():
            neuron.weights -= 0.01 * neuron.dweights
            neuron.bias -= 0.01 * neuron.dbias
            neuron.weights.grad.zero_()
            neuron.bias.grad.zero_()
    
    if epoch % 30 == 0:
        print(f"Epoch {epoch}: {loss.loss}")

print(neuron.weights.T, neuron.bias)
print(coef)

Epoch 0: 115.4983901977539
Epoch 30: 1.0027179087046534e-10
Epoch 60: 1.0027179087046534e-10
Epoch 90: 1.0027179087046534e-10
tensor([[98.5576, 56.5522, 97.1467, 21.5002]], grad_fn=<PermuteBackward0>) tensor([[0.5000]], requires_grad=True)
[98.55761921 56.552178   97.14665951 21.50022183]


3.1.2 Воспользовавшись классами `Linear` и `MSELoss` из задачи 2.1.4 и 2.3.1, `ReLU` из 2.2.1 и автоматическим дифференцированием, которое предоставляет `torch`, решить задачу регрессии. Для оптимизации использовать пакетный градиентный спуск. Вывести график функции потерь в зависимости от номера эпохи. Вывести на одном графике исходные данные и предсказанные значения.

In [48]:
class Linear:
    def __init__(self, n_features, n_neurons): 
        self.weights = torch.randn(n_features, n_neurons, requires_grad=True)
        self.biases = torch.randn(n_neurons, requires_grad=True)
 
    def forward(self, inputs):
        return inputs @ self.weights + self.biases # <реализовать логику слоя>

    def backward(self):
        self.dweights = self.weights.grad
        self.dbiases = self.biases.grad


class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs.clamp(min=0)
        return self.output

    def backward(self):
        self.dinputs = self.inputs.clamp(min=0)

class MSELoss:
    def forward(self, y_pred, y_true):
        self.loss = (y_pred.flatten() - y_true.flatten()).pow(2).mean()
        return self.loss

    def backward(self):
        self.loss.backward()

In [49]:
X, _, coef = make_regression(n_features=1, n_informative=1, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
# y = torch.from_numpy(y).to(dtype=torch.float32)
y = torch.sin(X)

In [50]:
fc1 = Linear(1, 512)
relu1 = Activation_ReLU()
fc2 = Linear(512, 1)

loss = MSELoss()
lr = 0.001
epochs = 20001

ys = []
losses = []
for epoch in range(epochs):
    out = fc2.forward(relu1.forward(fc1.forward(X)))
    data_loss = loss.forward(out, y)
    losses.append(data_loss.detach())
    
    if epoch % 2000 == 0:
        print(f'epoch {epoch} mean loss {data_loss}')
        ys.append(out.flatten())

    loss.backward()
    fc2.backward()
    fc1.backward()

    with torch.no_grad():
        fc1.weights -= lr * fc1.dweights
        fc1.biases -= lr * fc1.dbiases
        fc2.weights -= lr * fc2.dweights
        fc2.biases -= lr * fc2.dbiases

        fc1.weights.grad.zero_()
        fc1.biases.grad.zero_()
        fc2.weights.grad.zero_()
        fc2.biases.grad.zero_()
        
        

epoch 0 mean loss 384.0694274902344
epoch 2000 mean loss 0.003967610187828541
epoch 4000 mean loss 0.0015943781472742558
epoch 6000 mean loss 0.0009283197578042746
epoch 8000 mean loss 0.0006128075183369219
epoch 10000 mean loss 0.0004258542612660676
epoch 12000 mean loss 0.0003247694403398782
epoch 14000 mean loss 0.000262875750195235
epoch 16000 mean loss 0.00021854515944141895
epoch 18000 mean loss 0.00018552588880993426
epoch 20000 mean loss 0.00016058824257925153


In [51]:
px.line(x=range(epochs), y=losses)

In [52]:
for out in ys:
    fig = px.scatter(x = X.flatten().numpy(), y = y.flatten().numpy())
    fig.add_scatter(x = X.flatten().numpy(), y = out.detach().numpy(), mode='markers')
    fig.show()

## 3.2 Алгоритмы оптимизации в `torch.optim`

3.2.1 Решить задачу 3.1.1, воспользовавшись оптимизатором `optim.SDG` для применения стохастического градиентого спуска

In [53]:
X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)

neuron = Neuron(4)
loss = SquaredLoss()
optimizer = torch.optim.SGD([neuron.weights, neuron.bias], lr=0.01)

for epoch in range(100):
    for x_example, y_example in zip(X, y):
        y_pred = neuron.forward(x_example)
        loss.forward(y_pred, y_example)
        loss.backward()
        neuron.backward()
        with torch.no_grad():
            optimizer.step()
            neuron.weights.grad.zero_()
            neuron.bias.grad.zero_()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}: {loss.loss}")

print(neuron.weights.T.data.numpy(), neuron.bias.data.numpy())
print(coef)

Epoch 0: 1.9065592288970947
Epoch 20: 2.6284396881237626e-10
Epoch 40: 2.6284396881237626e-10
Epoch 60: 2.6284396881237626e-10
Epoch 80: 2.6284396881237626e-10
[[63.99593  42.961056 17.899776 94.14303 ]] [[0.5000021]]
[63.99593703 42.96105991 17.89976908 94.14305841]


3.2.2 Решить задачу 3.1.2, воспользовавшись оптимизатором `optim.Adam` для применения пакетного градиентого спуска. Вывести график функции потерь в зависимости от номера эпохи. Вывести на одном графике исходные данные и предсказанные значения.

In [54]:
X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
X = torch.from_numpy(X).to(dtype=torch.float32)
y = torch.from_numpy(y).to(dtype=torch.float32)
y = y / y.abs().max()  # Сходимость очень долгая если значения y большие 

neuron = Neuron(4)
loss = SquaredLoss()
optimizer = torch.optim.Adam([neuron.weights, neuron.bias])

for epoch in range(100):
    for x_example, y_example in zip(X, y):
        y_pred = neuron.forward(x_example)
        loss.forward(y_pred, y_example)
        loss.backward()
        neuron.backward()
        with torch.no_grad():
            optimizer.step()
            neuron.weights.grad.zero_()
            neuron.bias.grad.zero_()
    
    if epoch % 20 == 0:
        print(f"Epoch {epoch}: {loss.loss}")

print(neuron.weights.T.data.numpy(), neuron.bias.data.numpy())
print(coef)

Epoch 0: 0.037800874561071396
Epoch 20: 0.005875013768672943
Epoch 40: 0.00030034768860787153
Epoch 60: 8.643594728852122e-10
Epoch 80: 9.792167077193881e-14
[[0.1710373  0.07236549 0.02167697 0.28899625]] [[0.00170555]]
[50.14254846 21.21518825  6.35497211 84.72416954]


## 3.3 Построение сетей при помощи `torch.nn`

3.3.1 Решить задачу регрессии, соблюдая следующие условия:

1. Оформить нейронную сеть в виде класса - наследника `nn.Module`
2. При создании сети использовать готовые блоки из `torch.nn`: слои, функции активации, функции потерь и т.д.
3. Для оптимизации использовать любой алгоритм оптимизации из `torch.optim` 

In [55]:
X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())

In [56]:
class Model331(torch.nn.Module):
    def __init__(self, input_size: int = 1, hidden_size: int = 512, output_size: int = 1):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.activation1 = torch.nn.ReLU()
        self.fc2 = torch.nn.Linear(hidden_size, output_size)
        
        
    def forward(self, x):
        x = self.fc1(x)
        x = self.activation1(x)
        x = self.fc2(x)
        return x

    
model311 = Model331()
optimizer = torch.optim.SGD(model311.parameters(), 0.01)
loss = torch.nn.MSELoss()

for epoch in range(10000):

    y_pred = model311(X)
    data_loss = loss(y_pred.flatten(), y.flatten())
    data_loss.backward()

    with torch.no_grad():
        optimizer.step()
    
    model311.zero_grad()
            
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}: {data_loss.item()}")


Epoch 0: 0.5161110758781433
Epoch 500: 0.13874846696853638
Epoch 1000: 0.0894254669547081
Epoch 1500: 0.05363546684384346
Epoch 2000: 0.031573716551065445
Epoch 2500: 0.01912643201649189
Epoch 3000: 0.012358319945633411
Epoch 3500: 0.008687042631208897
Epoch 4000: 0.006637268699705601
Epoch 4500: 0.0054061817936599255
Epoch 5000: 0.004582562018185854
Epoch 5500: 0.003965619020164013
Epoch 6000: 0.003450667019933462
Epoch 6500: 0.0030273383017629385
Epoch 7000: 0.0026766739320009947
Epoch 7500: 0.002394641051068902
Epoch 8000: 0.0021667645778506994
Epoch 8500: 0.0019789128564298153
Epoch 9000: 0.0018282068194821477
Epoch 9500: 0.0017072551418095827


In [57]:
px.scatter(x = X.flatten(), y = [model311(X).flatten().detach(), y.flatten()])

3.3.2 Решить задачу регрессии, соблюдая следующие условия:

1. Оформить нейронную сеть в виде объекта `nn.Sequential`
2. При создании сети использовать готовые блоки из `torch.nn`: слои, функции активации, функции потерь и т.д.
3. Для оптимизации использовать любой алгоритм оптимизации из `torch.optim` 

In [58]:
X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())

In [59]:
model_seq = torch.nn.Sequential(
    torch.nn.Linear(1, 512),
    torch.nn.ReLU(),
    torch.nn.Linear(512, 1)
)
optimizer = torch.optim.SGD(model_seq.parameters(), 0.01)
loss = torch.nn.MSELoss()

for epoch in range(10000):

    y_pred = model_seq(X)
    data_loss = loss(y_pred, y)
    data_loss.backward()

    with torch.no_grad():
        optimizer.step()
        model_seq.zero_grad()
            
    
    if epoch % 500 == 0:
        print(f"Epoch {epoch}: {data_loss.item()}")


Epoch 0: 0.6019578576087952
Epoch 500: 0.1303716003894806
Epoch 1000: 0.08046234399080276
Epoch 1500: 0.046695731580257416
Epoch 2000: 0.026943426579236984
Epoch 2500: 0.0163160040974617
Epoch 3000: 0.010783165693283081
Epoch 3500: 0.007870141416788101
Epoch 4000: 0.0062606860883533955
Epoch 4500: 0.0052856020629405975
Epoch 5000: 0.004616457968950272
Epoch 5500: 0.004087779670953751
Epoch 6000: 0.0036317554768174887
Epoch 6500: 0.003219535341486335
Epoch 7000: 0.0028528340626507998
Epoch 7500: 0.0025437527801841497
Epoch 8000: 0.0022832376416772604
Epoch 8500: 0.002073063515126705
Epoch 9000: 0.001897733542136848
Epoch 9500: 0.0017503839917480946


In [60]:
px.scatter(x = X.flatten(), y = [model_seq(X).flatten().detach(), y.flatten()])

## 3.4. Datasets and dataloaders

In [61]:
from torch.utils.data import Dataset, DataLoader

3.4.1 Создать датасет, поставляющий данные из задачи 3.1.2. 

Создать `DataLoader` на основе этого датасета и проверить работоспособность.

Воспользовавшись результатами 3.3.1 (или 3.3.2) обучите модель, пользуясь мини-пакетным градиентным спуском с размером пакета (`batch_size`) = 10

In [62]:

class SinDataset(Dataset):
    def __init__(self, inp: torch.Tensor, target: torch.Tensor):
        self.inp, self.tgt = inp, target

    def __len__(self):
        return self.inp.shape[0]

    def __getitem__(self, idx):
        return self.inp[idx], self.tgt[idx]


X = torch.linspace(0, 1, 100).view(-1, 1)
y = torch.sin(2 * np.pi * X) + 0.1 * torch.rand(X.size())
dataset = SinDataset(X, y)


model = Model331()
optimizer = torch.optim.SGD(model.parameters(), 0.01)
loss = torch.nn.MSELoss()

for i in range(1001):
    for x, y in DataLoader(dataset, batch_size=10):
        y_pred = model(x)
        data_loss = loss(y_pred.flatten(), y.flatten())
        data_loss.backward()


        optimizer.step()
        optimizer.zero_grad()

    if i % 500 == 0:
        print(f"Epoch {i}: {data_loss.item()}")


Epoch 0: 19.219745635986328
Epoch 500: 0.006963616702705622
Epoch 1000: 0.002120424760505557


3.4.2 Предсказание цен алмазов

3.4.2.1 Создайте датасет на основе файла diamonds.csv. 

1. Удалите все нечисловые столбцы
2. Целевой столбец (`y`) - `price`
3. Преобразуйте данные в тензоры корректных размеров

3.4.2.2 Разбейте датасет на обучающий и тестовый датасет при помощи `torch.utils.data.random_split`.

3.4.2.3 Обучите модель для предсказания цен при помощи мини-пакетного градиентного спуска (`batch_size = 256`). 

3.4.2.4 Выведите график функции потерь в зависимости от номера эпохи (значение потерь для эпохи рассчитывайте как среднее значение ошибок на каждом батче). Проверьте качество модели на тестовой выборке. 


In [63]:
import pandas as pd

# 3.4.2.1
class DiamondsDataset(Dataset):
    def __init__(self, data: pd.DataFrame, target: pd.Series):
        self.data = torch.tensor(data.values).float()
        self.target = torch.tensor(target.values).float()

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        return self.data[idx], self.target[idx]

data = pd.read_csv("diamonds.csv", index_col=0)
dataset = DiamondsDataset(data.drop(columns=["cut", "color", "clarity", "price"]), data.price)


In [64]:
data

,carat,cut,color,clarity,depth,table,price,x,y,z
1,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
2,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
3,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
4,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
5,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75
...,...,...,...,...,...,...,...,...,...,...
53936,0.72,Ideal,D,SI1,60.8,57.0,2757,5.75,5.76,3.50
53937,0.72,Good,D,SI1,63.1,55.0,2757,5.69,5.75,3.61
53938,0.70,Very Good,D,SI1,62.8,60.0,2757,5.66,5.68,3.56
53939,0.86,Premium,H,SI2,61.0,58.0,2757,6.15,6.12,3.74


In [65]:
# 3.4.2.2
test_size = int(len(dataset) * 0.2)
train_data, test_data = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

In [66]:
# Данные очень большие из-за этого числа выходят за пределы float32 и возникает nan.
# Построим модель с батч-нормализацией, чтобы избежать этого.

In [67]:
model = torch.nn.Sequential(
    torch.nn.BatchNorm1d(6),
    torch.nn.Linear(6, 256),
    torch.nn.ReLU(),
    torch.nn.Linear(256, 1)
)

In [68]:
# 3.4.2.3
optimizer = torch.optim.Adam(model.parameters(), 0.001)
loss = torch.nn.MSELoss()
losses = []


for i in range(51):
    epoch_loss = 0
    for j, (x, y) in enumerate(DataLoader(train_data, batch_size=256), 1):
        y_pred = model(x)
        data_loss = loss(y_pred.flatten(), y.flatten())
        data_loss.backward()
        epoch_loss += data_loss.item()
        optimizer.step()
        optimizer.zero_grad()

    losses.append(epoch_loss / j)
    if i % 10 == 0:
        print(f"Epoch {i}: {epoch_loss / j:.4f}")


Epoch 0: 31363964.1183
Epoch 10: 2461589.4800
Epoch 20: 2324450.4386
Epoch 30: 2296236.4423
Epoch 40: 2277311.7027
Epoch 50: 2257486.6257


In [69]:
px.line(x=range(51), y=losses, labels={"x": "Epoch", "y": "Loss"})

In [70]:
# 3.4.2.4
model.eval()
test_pred = model(test_data[:][0])
test_loss = loss(test_pred.flatten(), test_data[:][1].flatten()).item()
print(f"Test loss: {test_loss:.4f}")

Test loss: 2472626.0000


3.4.3 Модифицируйте метод `__init__` датасета из 3.4.2 таким образом, чтобы он мог принимать параметр `transform: callable`. Реализуйте класс `DropColsTransform` для удаления нечисловых данных из массива. Реализуйте класс `ToTensorTransorm` для трансформации массива в тензор.

In [81]:
class DiamondsDataset(Dataset):
    def __init__(self, data, transform: callable = None):
        self.data = data
        self.X = data.drop(columns=["price"])
        self.transform = transform
        self.y = data.y

    def __len__(self):
        return self.data.shape[0]

    def __getitem__(self, idx):
        sample = self.X.iloc[idx], self.y.iloc[idx]
        if self.transform:
            sample = self.transform(sample)
        return sample

In [82]:
class DropColsTransform:
    def __init__(self, drop: list):
        self.drop = drop

    def __call__(self, sample):
        _X, _y = sample
        # <удаление из X столбцов self.drop>
        _X: pd.Series
        _X = _X.drop(self.drop)
        return _X.values, _y

In [83]:
class ToTensorTransform:
    def __call__(self, sample):
        _X, _y = sample
        # <преобразование X и y в тензоры>
        return torch.tensor(list(_X)).float(), torch.tensor(_y).float()

In [84]:

from torchvision import transforms

drop = DropColsTransform(drop=["cut", "color", "clarity"])
to_tensor = ToTensorTransform()
dataset = DiamondsDataset(data, transforms.Compose([drop, to_tensor]))
train_data, test_data = torch.utils.data.random_split(dataset, [len(dataset) - test_size, test_size])

In [85]:
# 3.4.2.3
model = torch.nn.Sequential(
    torch.nn.BatchNorm1d(6),
    torch.nn.Linear(6, 128),
    torch.nn.ReLU(),
    torch.nn.Linear(128, 1)
)

optimizer = torch.optim.Adam(model.parameters(), 0.01)
loss = torch.nn.MSELoss()
losses = []


for i in range(11):
    epoch_loss = 0
    for j, (x, y) in enumerate(DataLoader(train_data, batch_size=4096), 1):
        y_pred = model(x)
        data_loss = loss(y_pred.flatten(), y.flatten())
        data_loss.backward()
        epoch_loss += data_loss.item()

        with torch.no_grad():
            optimizer.step()
            model.zero_grad()

    losses.append(epoch_loss / j)
    if i % 2 == 0:
        print(f"Epoch {i}: {epoch_loss / j:.4f}")

Epoch 0: 16.4085
Epoch 2: 1.9779
Epoch 4: 0.3294
Epoch 6: 0.0835
Epoch 8: 0.0500
Epoch 10: 0.0375
